<a href="https://colab.research.google.com/github/slin35/RobotProducer/blob/main/robotproducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np
from nltk.stem import WordNetLemmatizer

In [ ]:
# import data
data1 = pd.read_csv('/tmdb_5000_movies.csv')
data2 = pd.read_csv('/tmdb_5000_credits.csv')

data1 = data1[['genres', 'title', 'overview', 'id']]
data2 = data2[['movie_id', 'title', 'cast', 'crew']]

data = pd.merge(data1, data2, left_on='id', right_on='movie_id', how='inner')
data = data[['title_x', 'overview', 'genres', 'cast', 'crew']]
data.rename({'title_x': 'title'}, axis=1, inplace=True)



In [ ]:
# cleanup genres
data['genres'] = [json.loads(i) if i != [] else [] for i in data['genres']]
data['genres'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['genres']]

# cleanup cast
data['cast'] = [json.loads(i) if i != [] else [] for i in data['cast']]
data['cast'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['cast']]

# cleanup crew
data['crew'] = [json.loads(i) if i != [] else [] for i in data['crew']]
data['crew'] = [[j['name']  for j in i if 'job' in j and j['job'] == 'Director'] for i in data['crew']]


In [ ]:
# tokenize overview and remove stopwords and punctuation
'''
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

data['overview'] = [word_tokenize(line) if type(line) is str else [] for line in data['overview']]

data['overview'] = [[w for w in line if w not in stop_words and w not in string.punctuation] for line in data['overview']]

# lemmatize words

lemmatizer = WordNetLemmatizer()
data['overview'] = [[lemmatizer.lemmatize(w) for w in line] for line in data['overview']]
'''

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
!pip install bert-tensorflow

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)


TensorFlow 1.x selected.
1.15.2


In [12]:
import bert

from bert import run_classifier
from bert import optimization
from bert import bert_tokenization
from bert import modeling
from bert import run_classifier_with_tfhub

In [13]:
# BERT-based, uncased model: 12-layer, 768-hidden, 12-heads, 110M parematers

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])    
  return bert_tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
tokenizer.tokenize("this is my example")

['this', 'is', 'my', 'example']